In [21]:
import imutils
import numpy as np
import cv2

In [22]:
from tensorflow.keras.models import load_model
model1 = load_model('F:/Project_Mask_NoMask/model2.h5')

In [23]:
#!curl -O https://raw.githubusercontent.com/opencv/opencv/master/samples/dnn/face_detector/deploy.prototxt
#!curl -O https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

In [24]:
print("[INFO] loading model...")
prototxt = 'F:/Project_Mask_NoMask/deploy.prototxt'
model = 'F:/Project_Mask_NoMask/res10_300x300_ssd_iter_140000.caffemodel'
net = cv2.dnn.readNetFromCaffe(prototxt, model)

[INFO] loading model...


In [25]:
import cv2

 

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    if ret:

        # Excute frame here
        #frame = cv2.imread(frame)
        
        frame = imutils.resize(frame, width=600)
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                text = "{:.2f}%".format(confidence * 100)
                y = startY - 10 if startY - 10 > 10 else startY + 10
                
                minX, maxX = min(startX, endX), max(startX, endX)
                minY, maxY = min(startY, endY), max(startY, endY)
                face = frame[minY:maxY, minX:maxX].copy()
                img = cv2.resize(face,(128,128))
                img = np.array([img/255.])
                y_hat = model1.predict(img)
                predic = np.argmax(y_hat,axis=1)
                if predic == 0:
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
                    cv2.putText(frame,"No mask:", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                if predic ==1:
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
                    cv2.putText(frame,"Mask:", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
                if predic == 2:
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (255, 0, 0), 2)
                    cv2.putText(frame,"Incorect mask:", (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 2)
        
        cv2.imshow("frame", frame)
        
        key = cv2.waitKey(1)

        if key == 27:

            break

 

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 42ms/step
